In [19]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 시계열, LSTM

In [10]:
# 2. LSTM model
def lstm_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    return lstm

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

path = '/project/LSH/'
with tf.device('/device:GPU:0'):
    # 1. Data load
    x = np.load(path + 'x_(7727,10,4068).npy')
    y = np.load(path + 'y_(7727,1).npy')

    # 3-1. Best model saving
    MODEL_SAVE_FOLDER_PATH = './models/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []
    sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

    # 4. Crossvalidation
    for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
        X_train, y_train = x[train_index,:,:], y[train_index]
        X_test, y_test = x[test_index,:,:], y[test_index]

        # 3-2. Best model saving
        model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_loss:{val_loss:.4f}.hdf5'

        cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                        verbose=1, save_best_only=True)
        early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)


        model = lstm_model()
        model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                      loss = "binary_crossentropy", metrics=['acc'])
        model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                  callbacks=[early_stop, cb_checkpoint], shuffle=False)

        # 5. accuracy checking 
#         from pathlib import Path
#         paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
#         best_model_path = str(paths)
        best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
        print('############# model path : ', best_model_path)

        best_model = keras.models.load_model(best_model_path) 
#         y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
        y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

        y_pred_test[y_pred_test>0.5]=1
        y_pred_test[y_pred_test<=0.5]=0
        precision = precision_score(y_test, y_pred_test)
        recall = recall_score(y_test, y_pred_test)
        f1 = f1_score(y_test, y_pred_test)
        roc_auc = roc_auc_score(y_test, y_pred_test)
        acc = accuracy_score(y_test, y_pred_test)
        
        acc_list.append(acc)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(roc_auc)
        
    print(f'정확도 : {np.mean(acc_list)}, Precision : {np.mean(precision_list)}, Recall : {np.mean(recall_list)}, \
                      F1 : {np.mean(f1_list)}, roc_auc : {np.mean(auc_list)}')
    
    
    

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 7s 83ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_loss improved from inf to 0.67041, saving model to ./models/seed=0_01_val_loss:0.6704.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_loss improved from 0.67041 to 0.65412, saving model to ./models/seed=0_02_val_loss:0.6541.hdf5
Epoch 3/500
37/37 [==============================] - 3s 71ms/step - loss: 0.5885 - acc: 0.6785 - val_loss: 0.5440 - val_acc: 0.7296

Epoch 00003: val_loss improved from 0.65412 to 0.54401, saving model to ./models/seed=0_03_val_loss:0.5440.hdf5
Epoch 4/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4601 - acc: 0.7972 - val_loss: 0.5490 - val_acc: 0.7400

Epoch 00004: val_loss did not improve from 0.54401
Epoch 5/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4107 - ac


Epoch 00040: val_loss did not improve from 0.53963
Epoch 41/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0852 - acc: 0.9769 - val_loss: 1.1796 - val_acc: 0.7147

Epoch 00041: val_loss did not improve from 0.53963
Epoch 42/500
37/37 [==============================] - 3s 72ms/step - loss: 0.1201 - acc: 0.9612 - val_loss: 1.0783 - val_acc: 0.7180

Epoch 00042: val_loss did not improve from 0.53963
Epoch 43/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1880 - acc: 0.9361 - val_loss: 0.9264 - val_acc: 0.7296

Epoch 00043: val_loss did not improve from 0.53963
Epoch 44/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1414 - acc: 0.9525 - val_loss: 0.9126 - val_acc: 0.7257

Epoch 00044: val_loss did not improve from 0.53963
Epoch 45/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0728 - acc: 0.9838 - val_loss: 1.0125 - val_acc: 0.7206

Epoch 00045: val_loss did not improve from 0.53963
Epoch 46/500
37/37 [===

37/37 [==============================] - 3s 72ms/step - loss: 0.1287 - acc: 0.9633 - val_loss: 0.8432 - val_acc: 0.7419

Epoch 00021: val_loss did not improve from 0.48198
Epoch 22/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1189 - acc: 0.9657 - val_loss: 0.8716 - val_acc: 0.7426

Epoch 00022: val_loss did not improve from 0.48198
Epoch 23/500
37/37 [==============================] - 3s 72ms/step - loss: 0.1121 - acc: 0.9730 - val_loss: 0.9045 - val_acc: 0.7426

Epoch 00023: val_loss did not improve from 0.48198
Epoch 24/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1066 - acc: 0.9726 - val_loss: 0.9465 - val_acc: 0.7393

Epoch 00024: val_loss did not improve from 0.48198
Epoch 25/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1009 - acc: 0.9754 - val_loss: 0.9649 - val_acc: 0.7380

Epoch 00025: val_loss did not improve from 0.48198
Epoch 26/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1105 - acc:


Epoch 00002: val_loss improved from 0.66844 to 0.64736, saving model to ./models/seed=2_02_val_loss:0.6474.hdf5
Epoch 3/500
37/37 [==============================] - 3s 70ms/step - loss: 0.5979 - acc: 0.6781 - val_loss: 0.5097 - val_acc: 0.7691

Epoch 00003: val_loss improved from 0.64736 to 0.50973, saving model to ./models/seed=2_03_val_loss:0.5097.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4724 - acc: 0.7881 - val_loss: 0.4719 - val_acc: 0.7814

Epoch 00004: val_loss improved from 0.50973 to 0.47193, saving model to ./models/seed=2_04_val_loss:0.4719.hdf5
Epoch 5/500
37/37 [==============================] - 3s 71ms/step - loss: 0.3939 - acc: 0.8375 - val_loss: 0.4875 - val_acc: 0.7840

Epoch 00005: val_loss did not improve from 0.47193
Epoch 6/500
37/37 [==============================] - 3s 70ms/step - loss: 0.3550 - acc: 0.8585 - val_loss: 0.5500 - val_acc: 0.7329

Epoch 00006: val_loss did not improve from 0.47193
Epoch 7/500
37/37 [==========

...

37/37 [==============================] - 3s 71ms/step - loss: 0.0566 - acc: 0.9881 - val_loss: 1.1920 - val_acc: 0.7173

Epoch 00055: val_loss did not improve from 0.51110
Epoch 00055: early stopping
############# model path :  ./models/seed=6_05_val_loss:0.5111.hdf5
Epoch 1/500
37/37 [==============================] - 6s 83ms/step - loss: 0.6775 - acc: 0.5933 - val_loss: 0.6675 - val_acc: 0.6138

Epoch 00001: val_loss improved from inf to 0.66747, saving model to ./models/seed=7_01_val_loss:0.6675.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6803 - acc: 0.5976 - val_loss: 0.6660 - val_acc: 0.6138

Epoch 00002: val_loss improved from 0.66747 to 0.66600, saving model to ./models/seed=7_02_val_loss:0.6660.hdf5
Epoch 3/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6677 - acc: 0.6123 - val_loss: 0.6364 - val_acc: 0.6138

Epoch 00003: val_loss improved from 0.66600 to 0.63636, saving model to ./models/seed=7_03_val_loss:0.6364.hdf5
E

37/37 [==============================] - 3s 72ms/step - loss: 0.0902 - acc: 0.9778 - val_loss: 0.8959 - val_acc: 0.7581

Epoch 00035: val_loss did not improve from 0.49650
Epoch 36/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0878 - acc: 0.9786 - val_loss: 0.9052 - val_acc: 0.7568

Epoch 00036: val_loss did not improve from 0.49650
Epoch 37/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0840 - acc: 0.9812 - val_loss: 0.9221 - val_acc: 0.7568

Epoch 00037: val_loss did not improve from 0.49650
Epoch 38/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0786 - acc: 0.9814 - val_loss: 0.9407 - val_acc: 0.7555

Epoch 00038: val_loss did not improve from 0.49650
Epoch 39/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0796 - acc: 0.9810 - val_loss: 0.9468 - val_acc: 0.7549

Epoch 00039: val_loss did not improve from 0.49650
Epoch 40/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0739 - acc:


Epoch 00015: val_loss did not improve from 0.49233
Epoch 16/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1598 - acc: 0.9502 - val_loss: 0.7229 - val_acc: 0.7484

Epoch 00016: val_loss did not improve from 0.49233
Epoch 17/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1380 - acc: 0.9603 - val_loss: 0.7722 - val_acc: 0.7458

Epoch 00017: val_loss did not improve from 0.49233
Epoch 18/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1298 - acc: 0.9650 - val_loss: 0.8008 - val_acc: 0.7451

Epoch 00018: val_loss did not improve from 0.49233
Epoch 19/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1271 - acc: 0.9638 - val_loss: 0.8144 - val_acc: 0.7464

Epoch 00019: val_loss did not improve from 0.49233
Epoch 20/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1152 - acc: 0.9687 - val_loss: 0.9046 - val_acc: 0.7342

Epoch 00020: val_loss did not improve from 0.49233
Epoch 21/500
37/37 [===

Epoch 1/500
37/37 [==============================] - 6s 81ms/step - loss: 0.6713 - acc: 0.6153 - val_loss: 0.6805 - val_acc: 0.5899

Epoch 00001: val_loss improved from inf to 0.68054, saving model to ./models/seed=9_01_val_loss:0.6805.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6612 - acc: 0.6142 - val_loss: 0.6390 - val_acc: 0.5899

Epoch 00002: val_loss improved from 0.68054 to 0.63900, saving model to ./models/seed=9_02_val_loss:0.6390.hdf5
Epoch 3/500
37/37 [==============================] - 3s 70ms/step - loss: 0.5530 - acc: 0.7163 - val_loss: 0.5427 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.63900 to 0.54274, saving model to ./models/seed=9_03_val_loss:0.5427.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4425 - acc: 0.8082 - val_loss: 0.5441 - val_acc: 0.7484

Epoch 00004: val_loss did not improve from 0.54274
Epoch 5/500
37/37 [==============================] - 3s 71ms/step - loss: 0.3840 - ac


Epoch 00042: val_loss did not improve from 0.54274
Epoch 43/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0619 - acc: 0.9868 - val_loss: 1.1830 - val_acc: 0.7277

Epoch 00043: val_loss did not improve from 0.54274
Epoch 44/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0625 - acc: 0.9873 - val_loss: 1.1947 - val_acc: 0.7245

Epoch 00044: val_loss did not improve from 0.54274
Epoch 45/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0647 - acc: 0.9871 - val_loss: 1.2418 - val_acc: 0.7193

Epoch 00045: val_loss did not improve from 0.54274
Epoch 46/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0600 - acc: 0.9866 - val_loss: 1.2893 - val_acc: 0.7154

Epoch 00046: val_loss did not improve from 0.54274
Epoch 47/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0658 - acc: 0.9847 - val_loss: 1.2667 - val_acc: 0.7173

Epoch 00047: val_loss did not improve from 0.54274
Epoch 48/500
37/37 [===

- 정확도 df 만들고 평균 확인 (val_acc) : [0.75789133]
- 정확도 df 만들고 평균 확인 (val_loss) : [0.76073739]


In [11]:
df

,model,acc,precision,recall,f1,roc_auc
0,SVC(),0.771410,0.788014,0.856038,0.820570,0.747371
1,GaussianNB(),0.483635,0.763052,0.224153,0.346155,0.557342
2,MultinomialNB(),0.677490,0.748613,0.710381,0.728956,0.668147
3,BernoulliNB(),0.681759,0.768362,0.685593,0.724521,0.680670
4,KNeighborsClassifier(),0.659056,0.664553,0.891949,0.761593,0.592901
5,DecisionTreeClassifier(),0.659056,0.664553,0.891949,0.761593,0.592901
6,RandomForestClassifier(),0.747089,0.749543,0.879873,0.809475,0.709372
7,GradientBoostingClassifier(),0.748771,0.762184,0.855614,0.806143,0.718422
8,MLPClassifier(max_iter=1000),0.733571,0.780321,0.784746,0.782432,0.719034
9,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.732406,0.780573,0.781780,0.781042,0.718382


In [48]:
df

,0
0,0.758085
1,0.772962
2,0.756792
3,0.751617
4,0.774903
5,0.774256
6,0.773609
7,0.733506
8,0.760673
9,0.722510


In [27]:
# RNN model
def rnn_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=64, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=32, activation='relu', return_sequences=False))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1))
    return rnn

def gru_model(X_train):
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    gru = Sequential()
    gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    gru.add(GRU(units=128, activation='relu', return_sequences=True))
    gru.add(GRU(units=64, activation='relu', return_sequences=True))
    gru.add(GRU(units=32, activation='relu', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1))
    return gru

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

def bi_lstm_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(Bidirectional(LSTM(units=128, activation='hard_sigmoid', return_sequences=True)))
    lstm.add(Bidirectional(LSTM(units=64, activation='hard_sigmoid', return_sequences=True)))
    lstm.add(Dropout(0.2))
    lstm.add(Bidirectional(LSTM(units=64, activation='hard_sigmoid', return_sequences=True)))
    lstm.add(Bidirectional(LSTM(units=32, activation='hard_sigmoid', return_sequences=False)))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    return lstm

def bi_rnn_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    rnn.add(Bidirectional(SimpleRNN(units=128, activation='relu', return_sequences=True)))
    rnn.add(Bidirectional(SimpleRNN(units=64, activation='relu', return_sequences=True)))
    rnn.add(Bidirectional(SimpleRNN(units=32, activation='relu', return_sequences=False)))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1))
    return rnn

def bi_gru_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    gru = Sequential()
    gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    gru.add(Bidirectional(GRU(units=128, activation='relu', return_sequences=True)))
    gru.add(Bidirectional(GRU(units=64, activation='relu', return_sequences=True)))
    gru.add(Bidirectional(GRU(units=32, activation='relu', return_sequences=False)))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1))
    return gru

In [21]:
def time_series_cv(name):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,10,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        data={}
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]

            # 3-2. Best model saving
#             model_path = MODEL_SAVE_FOLDER_PATH + name +'{epoch:02d}_val_acc:{val_acc:.4f}.hdf5'

#             cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', verbose=1, save_best_only=True)
            early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)

            if name == 'LSTM':
                model = lstm_model()
            elif name == 'RNN':
                model = rnn_model()
            elif name == 'GRU':
                model = gru_model(X_train)
            elif name == 'Bi_LSTM':
                model = bi_lstm_model()
            elif name == 'Bi_RNN':
                model = bi_rnn_model()
            elif name == 'Bi_GRU':
                model = bi_gru_model()
                
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                      callbacks=[early_stop], shuffle=False)

#             5. accuracy checking 
            if name == 'LSTM':
                from pathlib import Path
                paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
                best_model_path = str(paths)
                best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
                print('############# model path : ', best_model_path)

                best_model = keras.models.load_model(best_model_path) 
                y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            else:    
                y_pred_test = model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0

            precision = precision_score(y_test, y_pred_test)
            recall = recall_score(y_test, y_pred_test)
            f1 = f1_score(y_test, y_pred_test)
            roc_auc = roc_auc_score(y_test, y_pred_test)
            acc = accuracy_score(y_test, y_pred_test)

            acc_list.append(acc)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            auc_list.append(roc_auc)

        print(f'정확도 : {np.mean(acc_list)}, Precision : {np.mean(precision_list)}, Recall : {np.mean(recall_list)}, \
                          F1 : {np.mean(f1_list)}, roc_auc : {np.mean(auc_list)}')   

In [22]:
rnn_df = time_series_cv('RNN')
rnn_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 2s 30ms/step - loss: 0.9806 - acc: 0.5795 - val_loss: 0.6695 - val_acc: 0.6352
Epoch 2/500
37/37 [==============================] - 1s 22ms/step - loss: 0.6184 - acc: 0.7096 - val_loss: 0.5996 - val_acc: 0.7044
Epoch 3/500
37/37 [==============================] - 1s 22ms/step - loss: 0.4626 - acc: 0.7955 - val_loss: 0.6521 - val_acc: 0.7160
Epoch 4/500
37/37 [==============================] - 1s 21ms/step - loss: 0.3726 - acc: 0.8516 - val_loss: 0.8737 - val_acc: 0.7387
Epoch 5/500
37/37 [==============================] - 1s 21ms/step - loss: 0.2991 - acc: 0.8990 - val_loss: 1.1217 - val_acc: 0.7393
Epoch 6/500
37/37 [==============================] - 1s 21ms/step - loss: 0.2382 - acc: 0.9200 - val_loss: 1.5821 - val_acc: 0.7380
Epoch 7/500
37/37 [==============================] - 1s 21ms/step - loss: 0.2486 - acc: 0.9072 - val_loss: 1.3799 - val_acc: 0.7257
Epoch 8/500
37/37 [==============================] - 1s 21ms/step - loss: 0.

37/37 [==============================] - 1s 24ms/step - loss: 0.0680 - acc: 0.9830 - val_loss: 2.0591 - val_acc: 0.7329
Epoch 11/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0488 - acc: 0.9864 - val_loss: 2.6331 - val_acc: 0.7283
Epoch 12/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0415 - acc: 0.9927 - val_loss: 2.5921 - val_acc: 0.7283
Epoch 13/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0403 - acc: 0.9937 - val_loss: 2.7314 - val_acc: 0.7277
Epoch 14/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0358 - acc: 0.9959 - val_loss: 2.7971 - val_acc: 0.7264
Epoch 15/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0400 - acc: 0.9929 - val_loss: 2.7141 - val_acc: 0.7322
Epoch 16/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0364 - acc: 0.9957 - val_loss: 2.8218 - val_acc: 0.7206
Epoch 17/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0312 

37/37 [==============================] - 1s 21ms/step - loss: 0.0605 - acc: 0.9957 - val_loss: 2.9746 - val_acc: 0.7316
Epoch 21/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0605 - acc: 0.9957 - val_loss: 3.0511 - val_acc: 0.7322
Epoch 22/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0599 - acc: 0.9961 - val_loss: 3.1146 - val_acc: 0.7309
Epoch 23/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0603 - acc: 0.9959 - val_loss: 3.1203 - val_acc: 0.7406
Epoch 24/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0597 - acc: 0.9961 - val_loss: 3.1907 - val_acc: 0.7380
Epoch 25/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0598 - acc: 0.9961 - val_loss: 3.0385 - val_acc: 0.7329
Epoch 26/500
37/37 [==============================] - 1s 20ms/step - loss: 0.0602 - acc: 0.9961 - val_loss: 3.0667 - val_acc: 0.7387
Epoch 27/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0569 

37/37 [==============================] - 1s 24ms/step - loss: 0.0406 - acc: 0.9972 - val_loss: 3.2252 - val_acc: 0.7290
Epoch 31/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0389 - acc: 0.9965 - val_loss: 3.0409 - val_acc: 0.7406
Epoch 32/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0614 - acc: 0.9929 - val_loss: 2.9532 - val_acc: 0.7199
Epoch 33/500
37/37 [==============================] - 1s 23ms/step - loss: 0.1684 - acc: 0.9726 - val_loss: 1.6621 - val_acc: 0.7173
Epoch 34/500
37/37 [==============================] - 1s 22ms/step - loss: 0.3407 - acc: 0.9090 - val_loss: 1.5690 - val_acc: 0.7342
Epoch 35/500
37/37 [==============================] - 1s 23ms/step - loss: 0.1739 - acc: 0.9504 - val_loss: 2.2343 - val_acc: 0.7348
Epoch 36/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0967 - acc: 0.9823 - val_loss: 2.4479 - val_acc: 0.7451
Epoch 37/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0838 

37/37 [==============================] - 1s 23ms/step - loss: 0.1129 - acc: 0.9724 - val_loss: 2.4163 - val_acc: 0.7225
Epoch 41/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0797 - acc: 0.9860 - val_loss: 2.4280 - val_acc: 0.7173
Epoch 42/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0633 - acc: 0.9868 - val_loss: 2.6314 - val_acc: 0.7160
Epoch 43/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0654 - acc: 0.9894 - val_loss: 2.7051 - val_acc: 0.7245
Epoch 44/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0477 - acc: 0.9931 - val_loss: 2.7144 - val_acc: 0.7251
Epoch 45/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0603 - acc: 0.9905 - val_loss: 2.7322 - val_acc: 0.7245
Epoch 46/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0515 - acc: 0.9946 - val_loss: 2.8987 - val_acc: 0.7212
Epoch 47/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0425 

37/37 [==============================] - 1s 23ms/step - loss: 0.0133 - acc: 0.9991 - val_loss: 3.2221 - val_acc: 0.7283
Epoch 50/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0135 - acc: 0.9991 - val_loss: 3.2734 - val_acc: 0.7316
Epoch 51/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0137 - acc: 0.9991 - val_loss: 3.3191 - val_acc: 0.7309
Epoch 52/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0135 - acc: 0.9991 - val_loss: 3.2703 - val_acc: 0.7348
Epoch 00052: early stopping
Epoch 1/500
37/37 [==============================] - 2s 30ms/step - loss: 1.0635 - acc: 0.5644 - val_loss: 0.6233 - val_acc: 0.6721
Epoch 2/500
37/37 [==============================] - 1s 23ms/step - loss: 0.5977 - acc: 0.7105 - val_loss: 0.6008 - val_acc: 0.7063
Epoch 3/500
37/37 [==============================] - 1s 23ms/step - loss: 0.4533 - acc: 0.8095 - val_loss: 0.8487 - val_acc: 0.7264
Epoch 4/500
37/37 [==============================] - 1s 2

37/37 [==============================] - 1s 23ms/step - loss: 0.1990 - acc: 0.9430 - val_loss: 1.6166 - val_acc: 0.7426
Epoch 7/500
37/37 [==============================] - 1s 23ms/step - loss: 0.1751 - acc: 0.9493 - val_loss: 2.1025 - val_acc: 0.7367
Epoch 8/500
37/37 [==============================] - 1s 22ms/step - loss: 0.1399 - acc: 0.9633 - val_loss: 2.3016 - val_acc: 0.7387
Epoch 9/500
37/37 [==============================] - 1s 22ms/step - loss: 0.2092 - acc: 0.9456 - val_loss: 1.7460 - val_acc: 0.7568
Epoch 10/500
37/37 [==============================] - 1s 22ms/step - loss: 0.3049 - acc: 0.9184 - val_loss: 1.3175 - val_acc: 0.7122
Epoch 11/500
37/37 [==============================] - 1s 21ms/step - loss: 0.2015 - acc: 0.9504 - val_loss: 1.3164 - val_acc: 0.7561
Epoch 12/500
37/37 [==============================] - 1s 22ms/step - loss: 0.1063 - acc: 0.9728 - val_loss: 1.6172 - val_acc: 0.7490
Epoch 13/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0693 - a

37/37 [==============================] - 1s 21ms/step - loss: 0.0635 - acc: 0.9905 - val_loss: 2.7296 - val_acc: 0.7361
Epoch 17/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0425 - acc: 0.9950 - val_loss: 2.7412 - val_acc: 0.7264
Epoch 18/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0344 - acc: 0.9970 - val_loss: 2.8453 - val_acc: 0.7316
Epoch 19/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0301 - acc: 0.9972 - val_loss: 2.9541 - val_acc: 0.7283
Epoch 20/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0294 - acc: 0.9978 - val_loss: 2.8494 - val_acc: 0.7270
Epoch 21/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0284 - acc: 0.9978 - val_loss: 2.9297 - val_acc: 0.7283
Epoch 22/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0280 - acc: 0.9981 - val_loss: 2.9175 - val_acc: 0.7335
Epoch 23/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0275 

37/37 [==============================] - 1s 23ms/step - loss: 0.0429 - acc: 0.9942 - val_loss: 2.4717 - val_acc: 0.7406
Epoch 26/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0476 - acc: 0.9937 - val_loss: 2.7209 - val_acc: 0.7316
Epoch 27/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0478 - acc: 0.9937 - val_loss: 2.6491 - val_acc: 0.7348
Epoch 28/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0408 - acc: 0.9957 - val_loss: 2.7863 - val_acc: 0.7354
Epoch 29/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0326 - acc: 0.9976 - val_loss: 2.8637 - val_acc: 0.7400
Epoch 30/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0320 - acc: 0.9968 - val_loss: 3.0272 - val_acc: 0.7348
Epoch 31/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0307 - acc: 0.9981 - val_loss: 3.0115 - val_acc: 0.7451
Epoch 32/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0310 

In [17]:
gru_df = time_series_cv('GRU')
gru_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 4s 57ms/step - loss: 1.0186 - acc: 0.5618 - val_loss: 0.6232 - val_acc: 0.6481
Epoch 2/500
37/37 [==============================] - 2s 46ms/step - loss: 0.5925 - acc: 0.6844 - val_loss: 0.6251 - val_acc: 0.6947
Epoch 3/500
37/37 [==============================] - 2s 46ms/step - loss: 0.5097 - acc: 0.7659 - val_loss: 0.8026 - val_acc: 0.7206
Epoch 4/500
37/37 [==============================] - 2s 45ms/step - loss: 0.4615 - acc: 0.8179 - val_loss: 1.0038 - val_acc: 0.7283
Epoch 5/500
37/37 [==============================] - 2s 45ms/step - loss: 0.3596 - acc: 0.8749 - val_loss: 1.5518 - val_acc: 0.7173
Epoch 6/500
37/37 [==============================] - 2s 46ms/step - loss: 0.3357 - acc: 0.9083 - val_loss: 2.6176 - val_acc: 0.6979
Epoch 7/500
37/37 [==============================] - 2s 46ms/step - loss: 0.3403 - acc: 0.8820 - val_loss: 1.4056 - val_acc: 0.7173
Epoch 8/500
37/37 [==============================] - 2s 46ms/step - loss: 0.

37/37 [==============================] - 2s 47ms/step - loss: 0.4553 - acc: 0.8004 - val_loss: 0.7068 - val_acc: 0.7523
Epoch 5/500
37/37 [==============================] - 2s 45ms/step - loss: 0.4398 - acc: 0.8125 - val_loss: 0.8335 - val_acc: 0.7031
Epoch 6/500
37/37 [==============================] - 2s 44ms/step - loss: 0.3816 - acc: 0.8233 - val_loss: 0.8350 - val_acc: 0.7245
Epoch 7/500
37/37 [==============================] - 2s 45ms/step - loss: 0.2785 - acc: 0.8872 - val_loss: 1.5137 - val_acc: 0.7458
Epoch 8/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1968 - acc: 0.9307 - val_loss: 1.7041 - val_acc: 0.7432
Epoch 9/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1511 - acc: 0.9525 - val_loss: 2.3716 - val_acc: 0.7439
Epoch 10/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1487 - acc: 0.9646 - val_loss: 2.6881 - val_acc: 0.7193
Epoch 11/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1504 - acc

Epoch 9/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1876 - acc: 0.9484 - val_loss: 2.2289 - val_acc: 0.7380
Epoch 10/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1335 - acc: 0.9737 - val_loss: 2.6809 - val_acc: 0.7342
Epoch 11/500
37/37 [==============================] - 2s 47ms/step - loss: 0.1228 - acc: 0.9799 - val_loss: 2.7442 - val_acc: 0.7316
Epoch 12/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1119 - acc: 0.9827 - val_loss: 2.7811 - val_acc: 0.7445
Epoch 13/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1124 - acc: 0.9819 - val_loss: 2.9760 - val_acc: 0.7354
Epoch 14/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1189 - acc: 0.9855 - val_loss: 2.9668 - val_acc: 0.7303
Epoch 15/500
37/37 [==============================] - 2s 47ms/step - loss: 0.1035 - acc: 0.9855 - val_loss: 3.1100 - val_acc: 0.7354
Epoch 16/500
37/37 [==============================] - 2s 45ms/step - l

37/37 [==============================] - 2s 44ms/step - loss: 0.1101 - acc: 0.9648 - val_loss: 2.4556 - val_acc: 0.7361
Epoch 14/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0765 - acc: 0.9776 - val_loss: 2.7570 - val_acc: 0.7393
Epoch 15/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0725 - acc: 0.9819 - val_loss: 3.0251 - val_acc: 0.7316
Epoch 16/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0722 - acc: 0.9830 - val_loss: 2.6440 - val_acc: 0.7490
Epoch 17/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0845 - acc: 0.9814 - val_loss: 2.7507 - val_acc: 0.7406
Epoch 18/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0753 - acc: 0.9855 - val_loss: 2.5826 - val_acc: 0.7516
Epoch 19/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0615 - acc: 0.9886 - val_loss: 2.8438 - val_acc: 0.7464
Epoch 20/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0560 

...

Epoch 25/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0627 - acc: 0.9931 - val_loss: 3.0841 - val_acc: 0.7354
Epoch 26/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0608 - acc: 0.9931 - val_loss: 3.2028 - val_acc: 0.7329
Epoch 27/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0587 - acc: 0.9937 - val_loss: 3.3058 - val_acc: 0.7322
Epoch 28/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0579 - acc: 0.9942 - val_loss: 3.2761 - val_acc: 0.7329
Epoch 29/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0654 - acc: 0.9914 - val_loss: 3.2879 - val_acc: 0.7283
Epoch 30/500
37/37 [==============================] - 2s 42ms/step - loss: 0.0680 - acc: 0.9912 - val_loss: 3.0838 - val_acc: 0.7296
Epoch 31/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0575 - acc: 0.9940 - val_loss: 3.1920 - val_acc: 0.7283
Epoch 32/500
37/37 [==============================] - 2s 44ms/step - 

Epoch 30/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1883 - acc: 0.9543 - val_loss: 2.3075 - val_acc: 0.7193
Epoch 31/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1125 - acc: 0.9715 - val_loss: 2.5231 - val_acc: 0.7426
Epoch 32/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0798 - acc: 0.9873 - val_loss: 2.6779 - val_acc: 0.7380
Epoch 33/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0818 - acc: 0.9879 - val_loss: 2.7122 - val_acc: 0.7361
Epoch 34/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0818 - acc: 0.9823 - val_loss: 2.6389 - val_acc: 0.7380
Epoch 35/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0757 - acc: 0.9894 - val_loss: 2.7354 - val_acc: 0.7497
Epoch 36/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0689 - acc: 0.9912 - val_loss: 2.8106 - val_acc: 0.7484
Epoch 37/500
37/37 [==============================] - 2s 43ms/step - 

Epoch 36/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0553 - acc: 0.9953 - val_loss: 3.2021 - val_acc: 0.7445
Epoch 37/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0623 - acc: 0.9937 - val_loss: 3.4109 - val_acc: 0.7361
Epoch 38/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0602 - acc: 0.9929 - val_loss: 3.3026 - val_acc: 0.7471
Epoch 39/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0580 - acc: 0.9944 - val_loss: 3.3124 - val_acc: 0.7451
Epoch 40/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0581 - acc: 0.9944 - val_loss: 3.3215 - val_acc: 0.7406
Epoch 41/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0552 - acc: 0.9953 - val_loss: 3.2963 - val_acc: 0.7393
Epoch 42/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0534 - acc: 0.9957 - val_loss: 3.3047 - val_acc: 0.7497
Epoch 43/500
37/37 [==============================] - 2s 45ms/step - 

Epoch 41/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0504 - acc: 0.9968 - val_loss: 3.1856 - val_acc: 0.7529
Epoch 42/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0510 - acc: 0.9965 - val_loss: 3.1697 - val_acc: 0.7516
Epoch 43/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0505 - acc: 0.9963 - val_loss: 3.1477 - val_acc: 0.7549
Epoch 44/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0504 - acc: 0.9965 - val_loss: 3.2085 - val_acc: 0.7497
Epoch 45/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0502 - acc: 0.9965 - val_loss: 3.0773 - val_acc: 0.7581
Epoch 46/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0498 - acc: 0.9965 - val_loss: 3.0765 - val_acc: 0.7633
Epoch 47/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0500 - acc: 0.9965 - val_loss: 3.0353 - val_acc: 0.7555
Epoch 48/500
37/37 [==============================] - 2s 45ms/step - 

Epoch 47/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0680 - acc: 0.9914 - val_loss: 2.8361 - val_acc: 0.7458
Epoch 48/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0610 - acc: 0.9937 - val_loss: 2.9788 - val_acc: 0.7439
Epoch 49/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0585 - acc: 0.9944 - val_loss: 2.9867 - val_acc: 0.7432
Epoch 50/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0570 - acc: 0.9953 - val_loss: 3.0586 - val_acc: 0.7419
Epoch 51/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0599 - acc: 0.9950 - val_loss: 3.0898 - val_acc: 0.7413
Epoch 52/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0590 - acc: 0.9953 - val_loss: 3.1290 - val_acc: 0.7406
Epoch 00052: early stopping
정확도 : 0.7441138421733506, Precision : 0.7872520637986647, Recall : 0.8004943502824861,                           F1 : 0.791396027057539, roc_auc : 0.7280987808998255


In [23]:
bi_lstm_df = time_series_cv('Bi_LSTM')
bi_lstm_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 12s 150ms/step - loss: 0.6761 - acc: 0.6056 - val_loss: 0.6703 - val_acc: 0.6054
Epoch 2/500
37/37 [==============================] - 5s 128ms/step - loss: 0.6716 - acc: 0.6065 - val_loss: 0.6616 - val_acc: 0.6054
Epoch 3/500
37/37 [==============================] - 5s 127ms/step - loss: 0.6045 - acc: 0.6675 - val_loss: 0.5589 - val_acc: 0.7206
Epoch 4/500
37/37 [==============================] - 5s 128ms/step - loss: 0.4837 - acc: 0.7799 - val_loss: 0.5224 - val_acc: 0.7464
Epoch 5/500
37/37 [==============================] - 5s 130ms/step - loss: 0.3950 - acc: 0.8306 - val_loss: 0.5505 - val_acc: 0.7387
Epoch 6/500
37/37 [==============================] - 5s 127ms/step - loss: 0.3374 - acc: 0.8634 - val_loss: 0.5919 - val_acc: 0.7426
Epoch 7/500
37/37 [==============================] - 5s 127ms/step - loss: 0.3124 - acc: 0.8766 - val_loss: 0.5885 - val_acc: 0.7348
Epoch 8/500
37/37 [==============================] - 5s 128ms/step -

Epoch 48/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0572 - acc: 0.9868 - val_loss: 1.1818 - val_acc: 0.7083
Epoch 49/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0589 - acc: 0.9864 - val_loss: 1.2386 - val_acc: 0.7038
Epoch 50/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0639 - acc: 0.9827 - val_loss: 1.1983 - val_acc: 0.7063
Epoch 51/500
37/37 [==============================] - 5s 129ms/step - loss: 0.0545 - acc: 0.9881 - val_loss: 1.2413 - val_acc: 0.7018
Epoch 52/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0516 - acc: 0.9879 - val_loss: 1.2466 - val_acc: 0.7012
Epoch 53/500
37/37 [==============================] - 5s 131ms/step - loss: 0.0533 - acc: 0.9881 - val_loss: 1.2209 - val_acc: 0.7005
Epoch 54/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0518 - acc: 0.9890 - val_loss: 1.2516 - val_acc: 0.7031
Epoch 00054: early stopping
Epoch 1/500
37/37 [===============

37/37 [==============================] - 5s 126ms/step - loss: 0.0681 - acc: 0.9840 - val_loss: 1.0764 - val_acc: 0.7329
Epoch 41/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0694 - acc: 0.9834 - val_loss: 1.0847 - val_acc: 0.7329
Epoch 42/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0673 - acc: 0.9836 - val_loss: 1.0874 - val_acc: 0.7335
Epoch 43/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0659 - acc: 0.9836 - val_loss: 1.1159 - val_acc: 0.7303
Epoch 44/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0673 - acc: 0.9834 - val_loss: 1.1081 - val_acc: 0.7335
Epoch 45/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0628 - acc: 0.9862 - val_loss: 1.1101 - val_acc: 0.7290
Epoch 46/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0611 - acc: 0.9871 - val_loss: 1.1166 - val_acc: 0.7303
Epoch 47/500
37/37 [==============================] - 5s 127ms/step - loss:

37/37 [==============================] - 5s 127ms/step - loss: 0.1109 - acc: 0.9698 - val_loss: 0.8450 - val_acc: 0.7490
Epoch 33/500
37/37 [==============================] - 5s 126ms/step - loss: 0.1076 - acc: 0.9709 - val_loss: 0.8584 - val_acc: 0.7432
Epoch 34/500
37/37 [==============================] - 5s 126ms/step - loss: 0.1045 - acc: 0.9724 - val_loss: 0.8833 - val_acc: 0.7419
Epoch 35/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0988 - acc: 0.9756 - val_loss: 0.9000 - val_acc: 0.7413
Epoch 36/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0964 - acc: 0.9752 - val_loss: 0.9179 - val_acc: 0.7439
Epoch 37/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0953 - acc: 0.9748 - val_loss: 0.9348 - val_acc: 0.7419
Epoch 38/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0910 - acc: 0.9776 - val_loss: 0.9413 - val_acc: 0.7400
Epoch 39/500
37/37 [==============================] - 5s 128ms/step - loss:

...

Epoch 1/500
37/37 [==============================] - 12s 149ms/step - loss: 0.6778 - acc: 0.6052 - val_loss: 0.6754 - val_acc: 0.5964
Epoch 2/500
37/37 [==============================] - 5s 128ms/step - loss: 0.6693 - acc: 0.6142 - val_loss: 0.6665 - val_acc: 0.5964
Epoch 3/500
37/37 [==============================] - 5s 127ms/step - loss: 0.6072 - acc: 0.6690 - val_loss: 0.5783 - val_acc: 0.7141
Epoch 4/500
37/37 [==============================] - 5s 125ms/step - loss: 0.4883 - acc: 0.7732 - val_loss: 0.5019 - val_acc: 0.7633
Epoch 5/500
37/37 [==============================] - 5s 128ms/step - loss: 0.4127 - acc: 0.8209 - val_loss: 0.5140 - val_acc: 0.7561
Epoch 6/500
37/37 [==============================] - 5s 125ms/step - loss: 0.3674 - acc: 0.8481 - val_loss: 0.5335 - val_acc: 0.7633
Epoch 7/500
37/37 [==============================] - 5s 126ms/step - loss: 0.3500 - acc: 0.8539 - val_loss: 0.6423 - val_acc: 0.7458
Epoch 8/500
37/37 [==============================] - 5s 128ms/step -

Epoch 1/500
37/37 [==============================] - 12s 149ms/step - loss: 0.6838 - acc: 0.5789 - val_loss: 0.6696 - val_acc: 0.6138
Epoch 2/500
37/37 [==============================] - 5s 129ms/step - loss: 0.6781 - acc: 0.5931 - val_loss: 0.6664 - val_acc: 0.6138
Epoch 3/500
37/37 [==============================] - 5s 125ms/step - loss: 0.6776 - acc: 0.5970 - val_loss: 0.6622 - val_acc: 0.6138
Epoch 4/500
37/37 [==============================] - 5s 127ms/step - loss: 0.6442 - acc: 0.6362 - val_loss: 0.5759 - val_acc: 0.7238
Epoch 5/500
37/37 [==============================] - 5s 127ms/step - loss: 0.5303 - acc: 0.7497 - val_loss: 0.5105 - val_acc: 0.7413
Epoch 6/500
37/37 [==============================] - 5s 126ms/step - loss: 0.4564 - acc: 0.8041 - val_loss: 0.5035 - val_acc: 0.7620
Epoch 7/500
37/37 [==============================] - 5s 128ms/step - loss: 0.4039 - acc: 0.8326 - val_loss: 0.5056 - val_acc: 0.7671
Epoch 8/500
37/37 [==============================] - 5s 130ms/step -

Epoch 54/500
37/37 [==============================] - 5s 129ms/step - loss: 0.0833 - acc: 0.9765 - val_loss: 0.9662 - val_acc: 0.7549
Epoch 55/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0980 - acc: 0.9769 - val_loss: 1.0452 - val_acc: 0.7510
Epoch 56/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0802 - acc: 0.9789 - val_loss: 1.0422 - val_acc: 0.7510
Epoch 00056: early stopping
Epoch 1/500
37/37 [==============================] - 12s 149ms/step - loss: 0.6798 - acc: 0.5912 - val_loss: 0.6630 - val_acc: 0.6229
Epoch 2/500
37/37 [==============================] - 5s 128ms/step - loss: 0.6732 - acc: 0.6056 - val_loss: 0.6588 - val_acc: 0.6229
Epoch 3/500
37/37 [==============================] - 5s 127ms/step - loss: 0.6371 - acc: 0.6309 - val_loss: 0.5447 - val_acc: 0.7380
Epoch 4/500
37/37 [==============================] - 5s 127ms/step - loss: 0.5014 - acc: 0.7674 - val_loss: 0.5134 - val_acc: 0.7393
Epoch 5/500
37/37 [==================

37/37 [==============================] - 5s 129ms/step - loss: 0.0599 - acc: 0.9868 - val_loss: 1.0596 - val_acc: 0.7400
Epoch 45/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0587 - acc: 0.9875 - val_loss: 1.0749 - val_acc: 0.7374
Epoch 46/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0583 - acc: 0.9862 - val_loss: 1.0852 - val_acc: 0.7406
Epoch 47/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0554 - acc: 0.9883 - val_loss: 1.1016 - val_acc: 0.7342
Epoch 48/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0593 - acc: 0.9877 - val_loss: 1.0983 - val_acc: 0.7387
Epoch 49/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0585 - acc: 0.9873 - val_loss: 1.1041 - val_acc: 0.7342
Epoch 50/500
37/37 [==============================] - 5s 129ms/step - loss: 0.0572 - acc: 0.9875 - val_loss: 1.1213 - val_acc: 0.7342
Epoch 51/500
37/37 [==============================] - 5s 128ms/step - loss:

37/37 [==============================] - 5s 127ms/step - loss: 0.0998 - acc: 0.9741 - val_loss: 0.9742 - val_acc: 0.7387
Epoch 35/500
37/37 [==============================] - 5s 128ms/step - loss: 0.0997 - acc: 0.9722 - val_loss: 1.0115 - val_acc: 0.7316
Epoch 36/500
37/37 [==============================] - 5s 129ms/step - loss: 0.1001 - acc: 0.9735 - val_loss: 1.0411 - val_acc: 0.7290
Epoch 37/500
37/37 [==============================] - 5s 126ms/step - loss: 0.0955 - acc: 0.9748 - val_loss: 1.0121 - val_acc: 0.7316
Epoch 38/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0960 - acc: 0.9748 - val_loss: 0.9614 - val_acc: 0.7451
Epoch 39/500
37/37 [==============================] - 5s 129ms/step - loss: 0.0921 - acc: 0.9745 - val_loss: 0.9535 - val_acc: 0.7439
Epoch 40/500
37/37 [==============================] - 5s 127ms/step - loss: 0.0932 - acc: 0.9754 - val_loss: 0.9590 - val_acc: 0.7329
Epoch 41/500
37/37 [==============================] - 5s 129ms/step - loss:

In [24]:
bi_rnn_df = time_series_cv('Bi_RNN')
bi_rnn_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 4s 48ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 2/500
37/37 [==============================] - 1s 35ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 3/500
37/37 [==============================] - 1s 37ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 4/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 5/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 6/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 7/500
37/37 [==============================] - 1s 35ms/step - loss: 9.4447 - acc: 0.3877 - val_loss: 9.3388 - val_acc: 0.3946
Epoch 8/500
37/37 [==============================] - 1s 35ms/step - loss: 9.

37/37 [==============================] - 1s 36ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 12/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 13/500
37/37 [==============================] - 1s 35ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 14/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 15/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 16/500
37/37 [==============================] - 1s 34ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 17/500
37/37 [==============================] - 1s 34ms/step - loss: 9.4047 - acc: 0.3903 - val_loss: 9.4585 - val_acc: 0.3868
Epoch 18/500
37/37 [==============================] - 1s 36ms/step - loss: 9.4047 

37/37 [==============================] - 1s 36ms/step - loss: 0.0904 - acc: 0.9879 - val_loss: 2.8673 - val_acc: 0.7361
Epoch 22/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0666 - acc: 0.9916 - val_loss: 2.7443 - val_acc: 0.7393
Epoch 23/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0635 - acc: 0.9920 - val_loss: 2.8682 - val_acc: 0.7342
Epoch 24/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0685 - acc: 0.9905 - val_loss: 2.8794 - val_acc: 0.7374
Epoch 25/500
37/37 [==============================] - 1s 35ms/step - loss: 0.1137 - acc: 0.9702 - val_loss: 2.2974 - val_acc: 0.7257
Epoch 26/500
37/37 [==============================] - 1s 36ms/step - loss: 0.1434 - acc: 0.9661 - val_loss: 2.0010 - val_acc: 0.7290
Epoch 27/500
37/37 [==============================] - 1s 36ms/step - loss: 0.1316 - acc: 0.9767 - val_loss: 2.7912 - val_acc: 0.7329
Epoch 28/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0742 

37/37 [==============================] - 1s 37ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 31/500
37/37 [==============================] - 1s 35ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 32/500
37/37 [==============================] - 1s 35ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 33/500
37/37 [==============================] - 1s 36ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 34/500
37/37 [==============================] - 1s 37ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 35/500
37/37 [==============================] - 1s 34ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 36/500
37/37 [==============================] - 1s 37ms/step - loss: 9.3315 - acc: 0.3950 - val_loss: 9.6780 - val_acc: 0.3726
Epoch 37/500
37/37 [==============================] - 1s 35ms/step - loss: 9.3315 

37/37 [==============================] - 1s 39ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 41/500
37/37 [==============================] - 1s 36ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 42/500
37/37 [==============================] - 1s 34ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 43/500
37/37 [==============================] - 1s 35ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 44/500
37/37 [==============================] - 1s 36ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 45/500
37/37 [==============================] - 1s 35ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 46/500
37/37 [==============================] - 1s 37ms/step - loss: 9.5778 - acc: 0.3791 - val_loss: 8.9397 - val_acc: 0.4204
Epoch 47/500
37/37 [==============================] - 1s 37ms/step - loss: 9.5745 

37/37 [==============================] - 1s 35ms/step - loss: 0.0273 - acc: 0.9981 - val_loss: 3.1671 - val_acc: 0.7335
Epoch 51/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0268 - acc: 0.9983 - val_loss: 3.2116 - val_acc: 0.7303
Epoch 52/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0266 - acc: 0.9983 - val_loss: 3.1944 - val_acc: 0.7322
Epoch 00052: early stopping
Epoch 1/500
37/37 [==============================] - 4s 48ms/step - loss: 4.3812 - acc: 0.4882 - val_loss: 0.7416 - val_acc: 0.6643
Epoch 2/500
37/37 [==============================] - 1s 36ms/step - loss: 0.6354 - acc: 0.7020 - val_loss: 0.5913 - val_acc: 0.7193
Epoch 3/500
37/37 [==============================] - 1s 36ms/step - loss: 0.4457 - acc: 0.8013 - val_loss: 0.5969 - val_acc: 0.7458
Epoch 4/500
37/37 [==============================] - 1s 35ms/step - loss: 0.3094 - acc: 0.8867 - val_loss: 0.9287 - val_acc: 0.7510
Epoch 5/500
37/37 [==============================] - 1s 35

37/37 [==============================] - 1s 34ms/step - loss: 0.1188 - acc: 0.9713 - val_loss: 1.6651 - val_acc: 0.7477
Epoch 8/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0912 - acc: 0.9804 - val_loss: 1.8347 - val_acc: 0.7445
Epoch 9/500
37/37 [==============================] - 1s 37ms/step - loss: 0.0621 - acc: 0.9883 - val_loss: 2.2945 - val_acc: 0.7380
Epoch 10/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0606 - acc: 0.9905 - val_loss: 2.3837 - val_acc: 0.7322
Epoch 11/500
37/37 [==============================] - 1s 37ms/step - loss: 0.0581 - acc: 0.9940 - val_loss: 2.5317 - val_acc: 0.7400
Epoch 12/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0494 - acc: 0.9950 - val_loss: 2.5765 - val_acc: 0.7477
Epoch 13/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0441 - acc: 0.9968 - val_loss: 2.6311 - val_acc: 0.7484
Epoch 14/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0429 - 

37/37 [==============================] - 1s 36ms/step - loss: 0.0384 - acc: 0.9970 - val_loss: 2.8057 - val_acc: 0.7484
Epoch 17/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0374 - acc: 0.9974 - val_loss: 2.9311 - val_acc: 0.7451
Epoch 18/500
37/37 [==============================] - 1s 37ms/step - loss: 0.0349 - acc: 0.9976 - val_loss: 2.9170 - val_acc: 0.7516
Epoch 19/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0339 - acc: 0.9978 - val_loss: 2.9733 - val_acc: 0.7432
Epoch 20/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0333 - acc: 0.9978 - val_loss: 3.0319 - val_acc: 0.7432
Epoch 21/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0332 - acc: 0.9978 - val_loss: 3.0565 - val_acc: 0.7400
Epoch 22/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0330 - acc: 0.9978 - val_loss: 3.1116 - val_acc: 0.7393
Epoch 23/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0330 

37/37 [==============================] - 1s 36ms/step - loss: 0.0269 - acc: 0.9983 - val_loss: 3.0751 - val_acc: 0.7367
Epoch 26/500
37/37 [==============================] - 1s 34ms/step - loss: 0.0269 - acc: 0.9981 - val_loss: 3.0690 - val_acc: 0.7322
Epoch 27/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0264 - acc: 0.9983 - val_loss: 3.0929 - val_acc: 0.7354
Epoch 28/500
37/37 [==============================] - 1s 35ms/step - loss: 0.0267 - acc: 0.9983 - val_loss: 3.1418 - val_acc: 0.7335
Epoch 29/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0271 - acc: 0.9981 - val_loss: 3.1497 - val_acc: 0.7393
Epoch 30/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0266 - acc: 0.9983 - val_loss: 3.1503 - val_acc: 0.7354
Epoch 31/500
37/37 [==============================] - 1s 37ms/step - loss: 0.0270 - acc: 0.9983 - val_loss: 3.1024 - val_acc: 0.7387
Epoch 32/500
37/37 [==============================] - 1s 36ms/step - loss: 0.0267 

In [28]:
bi_gru_df = time_series_cv('Bi_GRU')
bi_gru_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 7s 93ms/step - loss: 1.1272 - acc: 0.5700 - val_loss: 0.6185 - val_acc: 0.6695
Epoch 2/500
37/37 [==============================] - 3s 79ms/step - loss: 0.5482 - acc: 0.7420 - val_loss: 0.5750 - val_acc: 0.7257
Epoch 3/500
37/37 [==============================] - 3s 77ms/step - loss: 0.4594 - acc: 0.8024 - val_loss: 0.6810 - val_acc: 0.7342
Epoch 4/500
37/37 [==============================] - 3s 79ms/step - loss: 0.3840 - acc: 0.8546 - val_loss: 0.7529 - val_acc: 0.7232
Epoch 5/500
37/37 [==============================] - 3s 79ms/step - loss: 0.3154 - acc: 0.8943 - val_loss: 1.4320 - val_acc: 0.7477
Epoch 6/500
37/37 [==============================] - 3s 79ms/step - loss: 0.2484 - acc: 0.9210 - val_loss: 2.2343 - val_acc: 0.7212
Epoch 7/500
37/37 [==============================] - 3s 79ms/step - loss: 0.2120 - acc: 0.9346 - val_loss: 2.0565 - val_acc: 0.7406
Epoch 8/500
37/37 [==============================] - 3s 79ms/step - loss: 0.

Epoch 52/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0892 - acc: 0.9845 - val_loss: 3.0137 - val_acc: 0.7193
Epoch 00052: early stopping
Epoch 1/500
37/37 [==============================] - 7s 102ms/step - loss: 1.2520 - acc: 0.5452 - val_loss: 0.6262 - val_acc: 0.6429
Epoch 2/500
37/37 [==============================] - 3s 79ms/step - loss: 0.5972 - acc: 0.6919 - val_loss: 0.6030 - val_acc: 0.7206
Epoch 3/500
37/37 [==============================] - 3s 79ms/step - loss: 0.4908 - acc: 0.7868 - val_loss: 0.6259 - val_acc: 0.7717
Epoch 4/500
37/37 [==============================] - 3s 80ms/step - loss: 0.4622 - acc: 0.8375 - val_loss: 0.7413 - val_acc: 0.7232
Epoch 5/500
37/37 [==============================] - 3s 79ms/step - loss: 0.3756 - acc: 0.8781 - val_loss: 1.3752 - val_acc: 0.7484
Epoch 6/500
37/37 [==============================] - 3s 79ms/step - loss: 0.3279 - acc: 0.8792 - val_loss: 1.5483 - val_acc: 0.7594
Epoch 7/500
37/37 [===========================

37/37 [==============================] - 3s 80ms/step - loss: 0.0469 - acc: 0.9970 - val_loss: 3.2824 - val_acc: 0.7387
Epoch 51/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0466 - acc: 0.9970 - val_loss: 3.2795 - val_acc: 0.7406
Epoch 52/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0465 - acc: 0.9970 - val_loss: 3.2995 - val_acc: 0.7413
Epoch 00052: early stopping
Epoch 1/500
37/37 [==============================] - 7s 91ms/step - loss: 1.1577 - acc: 0.5458 - val_loss: 0.6325 - val_acc: 0.6708
Epoch 2/500
37/37 [==============================] - 3s 79ms/step - loss: 0.6371 - acc: 0.6559 - val_loss: 0.6109 - val_acc: 0.6520
Epoch 3/500
37/37 [==============================] - 3s 79ms/step - loss: 0.5618 - acc: 0.7277 - val_loss: 0.6911 - val_acc: 0.7497
Epoch 4/500
37/37 [==============================] - 3s 79ms/step - loss: 0.4382 - acc: 0.8181 - val_loss: 0.6186 - val_acc: 0.7432
Epoch 5/500
37/37 [==============================] - 3s 78

37/37 [==============================] - 3s 77ms/step - loss: 0.0429 - acc: 0.9972 - val_loss: 3.2430 - val_acc: 0.7503
Epoch 49/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0430 - acc: 0.9972 - val_loss: 3.2502 - val_acc: 0.7523
Epoch 50/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0431 - acc: 0.9970 - val_loss: 3.2704 - val_acc: 0.7503
Epoch 51/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0471 - acc: 0.9968 - val_loss: 3.2456 - val_acc: 0.7516
Epoch 52/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0983 - acc: 0.9894 - val_loss: 2.8637 - val_acc: 0.7432
Epoch 00052: early stopping
Epoch 1/500
37/37 [==============================] - 7s 91ms/step - loss: 1.1939 - acc: 0.5454 - val_loss: 0.6805 - val_acc: 0.6779
Epoch 2/500
37/37 [==============================] - 3s 78ms/step - loss: 0.5754 - acc: 0.7049 - val_loss: 0.6278 - val_acc: 0.7290
Epoch 3/500
37/37 [==============================] - 3s 

...

37/37 [==============================] - 3s 78ms/step - loss: 0.0369 - acc: 0.9974 - val_loss: 3.2651 - val_acc: 0.7510
Epoch 42/500
37/37 [==============================] - 3s 80ms/step - loss: 0.0365 - acc: 0.9976 - val_loss: 3.2221 - val_acc: 0.7503
Epoch 43/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0363 - acc: 0.9976 - val_loss: 3.2557 - val_acc: 0.7555
Epoch 44/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0362 - acc: 0.9976 - val_loss: 3.2555 - val_acc: 0.7542
Epoch 45/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0363 - acc: 0.9976 - val_loss: 3.2703 - val_acc: 0.7549
Epoch 46/500
37/37 [==============================] - 3s 77ms/step - loss: 0.0363 - acc: 0.9976 - val_loss: 3.2868 - val_acc: 0.7536
Epoch 47/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0364 - acc: 0.9976 - val_loss: 3.2652 - val_acc: 0.7555
Epoch 48/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0362 

37/37 [==============================] - 3s 80ms/step - loss: 0.7320 - acc: 0.8723 - val_loss: 1.7656 - val_acc: 0.7096
Epoch 41/500
37/37 [==============================] - 3s 79ms/step - loss: 0.3718 - acc: 0.8973 - val_loss: 1.9553 - val_acc: 0.7413
Epoch 42/500
37/37 [==============================] - 3s 78ms/step - loss: 0.2086 - acc: 0.9443 - val_loss: 2.0881 - val_acc: 0.7380
Epoch 43/500
37/37 [==============================] - 3s 81ms/step - loss: 0.1298 - acc: 0.9644 - val_loss: 2.4135 - val_acc: 0.7380
Epoch 44/500
37/37 [==============================] - 3s 79ms/step - loss: 0.1024 - acc: 0.9732 - val_loss: 2.4837 - val_acc: 0.7348
Epoch 45/500
37/37 [==============================] - 3s 79ms/step - loss: 0.1012 - acc: 0.9778 - val_loss: 2.6335 - val_acc: 0.7432
Epoch 46/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0728 - acc: 0.9838 - val_loss: 2.7075 - val_acc: 0.7458
Epoch 47/500
37/37 [==============================] - 3s 81ms/step - loss: 0.0622 

37/37 [==============================] - 3s 78ms/step - loss: 0.6471 - acc: 0.9394 - val_loss: 3.1381 - val_acc: 0.7335
Epoch 39/500
37/37 [==============================] - 3s 78ms/step - loss: 0.7228 - acc: 0.8749 - val_loss: 1.4311 - val_acc: 0.7367
Epoch 40/500
37/37 [==============================] - 3s 80ms/step - loss: 0.2273 - acc: 0.9426 - val_loss: 1.8800 - val_acc: 0.7665
Epoch 41/500
37/37 [==============================] - 3s 78ms/step - loss: 0.1290 - acc: 0.9739 - val_loss: 2.2413 - val_acc: 0.7555
Epoch 42/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0984 - acc: 0.9821 - val_loss: 2.4241 - val_acc: 0.7536
Epoch 43/500
37/37 [==============================] - 3s 79ms/step - loss: 0.1035 - acc: 0.9845 - val_loss: 2.5705 - val_acc: 0.7600
Epoch 44/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0854 - acc: 0.9868 - val_loss: 2.5470 - val_acc: 0.7549
Epoch 45/500
37/37 [==============================] - 3s 79ms/step - loss: 0.2042 

37/37 [==============================] - 3s 80ms/step - loss: 0.0667 - acc: 0.9950 - val_loss: 2.9885 - val_acc: 0.7536
Epoch 37/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0655 - acc: 0.9948 - val_loss: 3.1194 - val_acc: 0.7490
Epoch 38/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0651 - acc: 0.9950 - val_loss: 3.1629 - val_acc: 0.7490
Epoch 39/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0639 - acc: 0.9959 - val_loss: 3.1681 - val_acc: 0.7561
Epoch 40/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0639 - acc: 0.9955 - val_loss: 3.1949 - val_acc: 0.7471
Epoch 41/500
37/37 [==============================] - 3s 80ms/step - loss: 0.0639 - acc: 0.9953 - val_loss: 3.2116 - val_acc: 0.7484
Epoch 42/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0636 - acc: 0.9957 - val_loss: 3.1992 - val_acc: 0.7490
Epoch 43/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0642 

37/37 [==============================] - 3s 79ms/step - loss: 0.0679 - acc: 0.9892 - val_loss: 2.8152 - val_acc: 0.7393
Epoch 35/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0620 - acc: 0.9916 - val_loss: 2.8514 - val_acc: 0.7439
Epoch 36/500
37/37 [==============================] - 3s 77ms/step - loss: 0.0574 - acc: 0.9937 - val_loss: 2.8912 - val_acc: 0.7400
Epoch 37/500
37/37 [==============================] - 3s 78ms/step - loss: 0.0556 - acc: 0.9937 - val_loss: 2.9938 - val_acc: 0.7432
Epoch 38/500
37/37 [==============================] - 3s 77ms/step - loss: 0.0542 - acc: 0.9953 - val_loss: 3.1318 - val_acc: 0.7451
Epoch 39/500
37/37 [==============================] - 3s 77ms/step - loss: 0.0561 - acc: 0.9950 - val_loss: 3.1886 - val_acc: 0.7445
Epoch 40/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0535 - acc: 0.9953 - val_loss: 3.1861 - val_acc: 0.7432
Epoch 41/500
37/37 [==============================] - 3s 79ms/step - loss: 0.0519 

# 비시계열

In [6]:
def non_time_series_cv(model):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:], y[train_index]
            X_test, y_test = x[test_index,:], y[test_index]
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)
            f1 = f1_score(y_test, pred)
            roc_auc = roc_auc_score(y_test, pred)
            acc = accuracy_score(y_test, pred)
            
            acc_list.append(acc)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            auc_list.append(roc_auc)
#             print(acc, precision, recall, f1, roc_auc)
        print(f'정확도 : {np.mean(acc_list)}, Precision : {np.mean(precision_list)}, Recall : {np.mean(recall_list)}, \
                      F1 : {np.mean(f1_list)}, roc_auc : {np.mean(auc_list)}')
        return [np.mean(acc_list), np.mean(precision_list), np.mean(recall_list), np.mean(f1_list), np.mean(auc_list)]

In [8]:
%%time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

result = []

# Support vector machine
model = SVC()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

# Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

# K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})

# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})
# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})
# Neural network
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(max_iter=1000)
temp = non_time_series_cv(model)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
temp = non_time_series_cv(model)
model = AdaBoostClassifier(n_estimators=500, 
                        random_state=10, 
                        learning_rate=0.1)
result.append({'model' : str(model), 'acc' : temp[0], 'precision' : temp[1],
               'recall' : temp[2], 'f1' : temp[3], 'roc_auc' : temp[4]})


0it [00:00, ?it/s]

정확도 : 0.7714100905562742, Precision : 0.7880143713559553, Recall : 0.8560381355932203,                       F1 : 0.8205702274167062, roc_auc : 0.7473712272650487


0it [00:00, ?it/s]

정확도 : 0.48363518758085383, Precision : 0.763051849775897, Recall : 0.22415254237288135,                       F1 : 0.3461551606026837, roc_auc : 0.5573420519173378


0it [00:00, ?it/s]

...

정확도 : 0.7487710219922381, Precision : 0.7621837385223997, Recall : 0.8556144067796609,                       F1 : 0.8061432712320394, roc_auc : 0.7184218213300297


0it [00:00, ?it/s]

정확도 : 0.7335705045278137, Precision : 0.7803205079860873, Recall : 0.7847457627118645,                       F1 : 0.7824320472113311, roc_auc : 0.7190340109240385


0it [00:00, ?it/s]

정확도 : 0.7324062095730919, Precision : 0.7805731264861739, Recall : 0.7817796610169492,                       F1 : 0.7810421937229945, roc_auc : 0.718381524860634
CPU times: user 9h 47min 9s, sys: 9h 16min 26s, total: 19h 3min 35s
Wall time: 36min 37s


In [9]:
# result.append({'model' : 'LSTM', 'mean' : 0.75789133})
df = pd.DataFrame(result)
df

,model,acc,precision,recall,f1,roc_auc
0,SVC(),0.771410,0.788014,0.856038,0.820570,0.747371
1,GaussianNB(),0.483635,0.763052,0.224153,0.346155,0.557342
2,MultinomialNB(),0.677490,0.748613,0.710381,0.728956,0.668147
3,BernoulliNB(),0.681759,0.768362,0.685593,0.724521,0.680670
4,KNeighborsClassifier(),0.659056,0.664553,0.891949,0.761593,0.592901
5,DecisionTreeClassifier(),0.659056,0.664553,0.891949,0.761593,0.592901
6,RandomForestClassifier(),0.747089,0.749543,0.879873,0.809475,0.709372
7,GradientBoostingClassifier(),0.748771,0.762184,0.855614,0.806143,0.718422
8,MLPClassifier(max_iter=1000),0.733571,0.780321,0.784746,0.782432,0.719034
9,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.732406,0.780573,0.781780,0.781042,0.718382


In [36]:
df.sort_values('mean')

,model,mean
1,GaussianNB(),0.483635
2,KNeighborsClassifier(),0.659056
5,MLPClassifier(max_iter=1000),0.732471
3,RandomForestClassifier(),0.738422
4,GradientBoostingClassifier(),0.748448
6,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.752199
7,LSTM,0.757891
0,SVC(),0.771410
